In [3]:
import pandas as pd
import json
from pandas import json_normalize

In [4]:
json_path = 'first.json'
with  open(json_path,'r') as file:
    data = json.load(file)


In [5]:
#creating dataframe for team players

players_data = []
for team, players in data["info"]["players"].items():
    for player in players:
        player_registry_id = data["info"]["registry"]["people"].get(player, None)
        players_data.append({"player": player, "team": team, "registry_id": player_registry_id})

players_df = pd.DataFrame(players_data)



In [6]:
meta_normalized = json_normalize(data['meta'])
innings_normalized = json_normalize(data['innings'])
info_normalized = json_normalize(data['info'])

In [7]:
info_df = pd.DataFrame.from_dict(info_normalized)

In [8]:
print(info_df)

   balls_per_over       city         dates gender match_type  overs  \
0               6  Bangalore  [2008-04-18]   male        T20     20   

  player_of_match   season team_type  \
0   [BB McCullum]  2007/08      club   

                                               teams  ...  \
0  [Royal Challengers Bangalore, Kolkata Knight R...  ...   

  registry.people.RT Ponting  registry.people.SB Joshi  \
0                   7d415ea5                  670709ec   

  registry.people.SC Ganguly registry.people.V Kohli  \
0                   725529bc                ba607b88   

  registry.people.VN Kulkarni registry.people.W Jaffer  \
0                    0b126008                 619aa81f   

  registry.people.WP Saha  registry.people.Z Khan toss.decision  \
0                fe11caa6                91a4a398         field   

                   toss.winner  
0  Royal Challengers Bangalore  

[1 rows x 50 columns]


In [9]:
# dropping use less data
columns_to_remove = [
    'players.Kolkata Knight Riders',
    'players.Royal Challengers Bangalore',
    'registry.people',
    'balls_per_over',
    'gender','season',
    'overs','event.name','match_type'
]

# Drop the specified columns
info_df = info_df.drop(columns=columns_to_remove, errors='ignore')


In [10]:
# removing all the attributes starting with registry 
columns_to_remove = [col for col in info_df.columns if col.startswith('registry')]
info_df_data= info_df.drop(columns=columns_to_remove)
print(info_df_data)

        city         dates player_of_match team_type  \
0  Bangalore  [2008-04-18]   [BB McCullum]      club   

                                               teams                  venue  \
0  [Royal Challengers Bangalore, Kolkata Knight R...  M Chinnaswamy Stadium   

   event.match_number officials.match_referees officials.reserve_umpires  \
0                   1              [J Srinath]             [VN Kulkarni]   

  officials.tv_umpires         officials.umpires  outcome.by.runs  \
0          [AM Saheba]  [Asad Rauf, RE Koertzen]              140   

          outcome.winner toss.decision                  toss.winner  
0  Kolkata Knight Riders         field  Royal Challengers Bangalore  


In [11]:
print(players_df)
players_df.head()

             player                         team registry_id
0        SC Ganguly        Kolkata Knight Riders    725529bc
1       BB McCullum        Kolkata Knight Riders    b8a55852
2        RT Ponting        Kolkata Knight Riders    7d415ea5
3         DJ Hussey        Kolkata Knight Riders    fd835ab3
4   Mohammad Hafeez        Kolkata Knight Riders    9ab63e7b
5         LR Shukla        Kolkata Knight Riders    b1451597
6           WP Saha        Kolkata Knight Riders    fe11caa6
7        AB Agarkar        Kolkata Knight Riders    fa463154
8          AB Dinda        Kolkata Knight Riders    66b30f71
9          M Kartik        Kolkata Knight Riders    abfeb126
10         I Sharma        Kolkata Knight Riders    5bb1a1c4
11         R Dravid  Royal Challengers Bangalore    0184dc35
12         W Jaffer  Royal Challengers Bangalore    619aa81f
13          V Kohli  Royal Challengers Bangalore    ba607b88
14        JH Kallis  Royal Challengers Bangalore    86dc8f2e
15         CL White  Roy

,player,team,registry_id
0,SC Ganguly,Kolkata Knight Riders,725529bc
1,BB McCullum,Kolkata Knight Riders,b8a55852
2,RT Ponting,Kolkata Knight Riders,7d415ea5
3,DJ Hussey,Kolkata Knight Riders,fd835ab3
4,Mohammad Hafeez,Kolkata Knight Riders,9ab63e7b


In [12]:
info_df_data = info_df_data.map(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x) 
# in this going through all the dataframe elements then we check whether value is  

In [13]:
info_df_data.head()

,city,dates,player_of_match,team_type,teams,venue,event.match_number,officials.match_referees,officials.reserve_umpires,officials.tv_umpires,officials.umpires,outcome.by.runs,outcome.winner,toss.decision,toss.winner
0,Bangalore,2008-04-18,BB McCullum,club,"Royal Challengers Bangalore, Kolkata Knight Ri...",M Chinnaswamy Stadium,1,J Srinath,VN Kulkarni,AM Saheba,"Asad Rauf, RE Koertzen",140,Kolkata Knight Riders,field,Royal Challengers Bangalore


 DATAFRAME FOR META

In [14]:
meta_df = pd.DataFrame.from_dict(meta_normalized)
meta_df.head()

,data_version,created,revision
0,1.0.0,2011-05-06,2


DATA FRAME FOR INNINGS

In [26]:
def extract_delivery_info(inning):
    inning_data = []
    for over_info in inning["overs"]:
        for delivery_info in over_info["deliveries"]:
            runs_info = delivery_info["runs"]
            wickets_info = delivery_info.get("wickets", None)
            wicket = 1 if wickets_info else None

            data = {
                "team": inning["team"],
                "over": over_info["over"],
                "batter": delivery_info["batter"],
                "bowler": delivery_info["bowler"],
                "non_striker": delivery_info["non_striker"],
                "batter_runs": runs_info["batter"],
                "extras_runs": runs_info["extras"],
                "total_runs": runs_info["total"],
                "wicket": wicket,
            }
            inning_data.append(data)
    return inning_data

# Flatten the data and create a DataFrame
match_data = []
for inning_data in data["innings"]:
    match_data.extend(extract_delivery_info(inning_data))

df = pd.DataFrame(match_data)

# Display the DataFrame
print()

                            team  over       batter     bowler  non_striker  \
0          Kolkata Knight Riders     0   SC Ganguly    P Kumar  BB McCullum   
1          Kolkata Knight Riders     0  BB McCullum    P Kumar   SC Ganguly   
2          Kolkata Knight Riders     0  BB McCullum    P Kumar   SC Ganguly   
3          Kolkata Knight Riders     0  BB McCullum    P Kumar   SC Ganguly   
4          Kolkata Knight Riders     0  BB McCullum    P Kumar   SC Ganguly   
..                           ...   ...          ...        ...          ...   
220  Royal Challengers Bangalore    14      P Kumar   I Sharma     SB Joshi   
221  Royal Challengers Bangalore    14     SB Joshi   I Sharma      P Kumar   
222  Royal Challengers Bangalore    14      P Kumar   I Sharma     SB Joshi   
223  Royal Challengers Bangalore    15     SB Joshi  LR Shukla      P Kumar   
224  Royal Challengers Bangalore    15     SB Joshi  LR Shukla      P Kumar   

     batter_runs  extras_runs  total_runs  wicket  

In [28]:
df.head(1)

,team,over,batter,bowler,non_striker,batter_runs,extras_runs,total_runs,wicket
0,Kolkata Knight Riders,0,SC Ganguly,P Kumar,BB McCullum,0,1,1,NaN
